In [1]:
import tensorflow as tf
import pickle
import numpy as np
import random

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Set parameters
image_size = 32
num_channels = 3
num_categories = 10
num_filters = 32
filter_size = 5
num_epochs = 200
batch_size = 10
num_batches = int(50000/batch_size)
keep_prob = 0.6

In [3]:
# Read CIFAR training data
train_data = None
train_labels = None
for file_index in range(5):
    train_file = open('data/cifar-10-batches-py/data_batch_' + str(file_index+1), 'rb')
    train_dict = pickle.load(train_file, encoding='latin1')
    train_file.close()
    if train_data is None:
        train_data = np.array(train_dict['data'], float)/255.0
        train_labels = train_dict['labels']
    else:
        train_data = np.concatenate((train_data, train_dict['data']), 0)
        train_labels = np.concatenate((train_labels, train_dict['labels']), 0)


In [4]:
print(np.shape(train_labels), np.shape(train_data))
print(train_labels)
print(train_data)

(50000,) (50000, 3072)
[6 9 9 ... 9 1 1]
[[2.31372549e-01 1.68627451e-01 1.96078431e-01 ... 5.49019608e-01
  3.29411765e-01 2.82352941e-01]
 [6.03921569e-01 4.94117647e-01 4.11764706e-01 ... 5.45098039e-01
  5.56862745e-01 5.64705882e-01]
 [1.00000000e+00 9.92156863e-01 9.92156863e-01 ... 3.25490196e-01
  3.25490196e-01 3.29411765e-01]
 ...
 [3.50000000e+01 4.00000000e+01 4.20000000e+01 ... 7.70000000e+01
  6.60000000e+01 5.00000000e+01]
 [1.89000000e+02 1.86000000e+02 1.85000000e+02 ... 1.69000000e+02
  1.71000000e+02 1.71000000e+02]
 [2.29000000e+02 2.36000000e+02 2.34000000e+02 ... 1.73000000e+02
  1.62000000e+02 1.61000000e+02]]


In [5]:
train_data = train_data.reshape([-1, num_channels, image_size, image_size])
print(np.shape(train_data))

(50000, 3, 32, 32)


In [6]:
train_data = train_data.transpose([0, 2, 3, 1])
print(np.shape(train_data))

(50000, 32, 32, 3)


In [7]:
train_labels = np.eye(num_categories)[train_labels]
print(np.shape(train_labels))
print(train_labels)

(50000, 10)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [8]:
# Read CIFAR test data
test_file = open('data/cifar-10-batches-py/test_batch', 'rb')
test_dict = pickle.load(test_file, encoding='latin1')
test_file.close()
test_data = test_dict['data']
test_labels = test_dict['labels']

In [9]:
# Preprocess test data and labels
test_data = test_data.reshape([-1, num_channels, image_size, image_size])
test_data = test_data.transpose([0, 2, 3, 1])
test_labels = np.eye(num_categories)[test_labels]

In [10]:
# Placeholders for CIFAR images
img_holder = tf.placeholder(tf.float32, [None, image_size, image_size,
num_channels])
lbl_holder = tf.placeholder(tf.float32, [None, num_categories])
train = tf.placeholder(tf.bool)

In [11]:
# Create convolution/pooling layers
conv1 = tf.layers.conv2d(img_holder, num_filters, filter_size, padding='same',
activation=tf.nn.relu)
print(np.shape(img_holder))
print(np.shape(conv1))

(?, 32, 32, 3)
(?, 32, 32, 32)


In [12]:
drop1 = tf.layers.dropout(conv1, keep_prob, training=train)
pool1 = tf.layers.max_pooling2d(drop1, 2, 2)
print(np.shape(pool1))
conv2 = tf.layers.conv2d(pool1, num_filters, filter_size, padding='same',
activation=tf.nn.relu)
drop2 = tf.layers.dropout(conv2, keep_prob, training=train)
pool2 = tf.layers.max_pooling2d(drop2, 2, 2)
print(np.shape(pool2))
conv3 = tf.layers.conv2d(pool2, num_filters, filter_size, padding='same',
activation=tf.nn.relu)
pool3 = tf.layers.max_pooling2d(conv3, 2, 2)
print(np.shape(pool3))
conv4 = tf.layers.conv2d(pool3, num_filters, filter_size, padding='same',
activation=tf.nn.relu)
drop3 = tf.layers.dropout(conv4, keep_prob, training=train)

(?, 16, 16, 32)
(?, 8, 8, 32)
(?, 4, 4, 32)


In [13]:
# Flatten input data
flatten = tf.reshape(drop3, [-1, 512])
print(np.shape(flatten))

(?, 512)


In [14]:
# Create connected layers
with tf.contrib.framework.arg_scope(
[tf.contrib.layers.fully_connected],
normalizer_fn=tf.contrib.layers.batch_norm,
normalizer_params={'is_training': train}):
    fc1 = tf.contrib.layers.fully_connected(flatten, 512)
    fc2 = tf.contrib.layers.fully_connected(fc1, num_categories,
    activation_fn=None)

In [15]:
# Compute loss
loss = tf.reduce_mean(
tf.nn.softmax_cross_entropy_with_logits(
logits=fc2, labels=lbl_holder))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [16]:
# Create optimizer
learning_rate = 0.0005
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
# Initialize variables
init = tf.global_variables_initializer()

In [17]:
# Determine success rate
prediction = tf.equal(tf.argmax(fc2, 1), tf.argmax(lbl_holder, 1))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

In [18]:
# Launch session
sess = tf.Session()
sess.run(init)
# Loop over epochs
for epoch in range(num_epochs):
    # Loop over batches
    for batch in range(num_batches):
        batch_start = random.randint(0, batch_size*(num_batches-1)-1)
        batch_end = batch_start + batch_size
        img_batch = train_data[batch_start:batch_end, :]
        lbl_batch = train_labels[batch_start:batch_end, :]
        sess.run(optimizer, feed_dict={img_holder: img_batch,
        lbl_holder: lbl_batch, train: True})
    print('epoch:', epoch+1)
    print('Accuracy: ', sess.run(accuracy, feed_dict={img_holder: test_data,
lbl_holder: test_labels, train: False}))
    

epoch: 1
Accuracy:  0.4034
epoch: 2
Accuracy:  0.4842
epoch: 3
Accuracy:  0.5441
epoch: 4
Accuracy:  0.5733
epoch: 5
Accuracy:  0.5772
epoch: 6
Accuracy:  0.5808


KeyboardInterrupt: 

In [42]:
print('Accuracy: ', sess.run(accuracy, feed_dict={img_holder: test_data,
lbl_holder: test_labels, train: False}))

Accuracy:  0.6526


In [78]:
print(random.randint(0, 0))

0
